In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Importing dependencies

In [1]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

### Load summarization pipeline

In [2]:
summarizer = pipeline('summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

### Get Blog Post

In [27]:
URL = "https://www.analyticsvidhya.com/blog/2023/08/fine-tuning-large-language-models/"

In [28]:
r = requests.get(URL)

In [29]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p'])
text = [result.text for result in results]
Article = ' '.join(text)

In [31]:
Article

'\n A Comprehensive Guide to Fine-Tuning Large Language Models Over the past few years, the landscape of natural language processing (NLP) has undergone a remarkable transformation, all thanks to the advent of large language models. These sophisticated models have opened the doors to a wide array of applications, ranging from language translation to sentiment analysis and even the creation of intelligent chatbots. But their versatility sets these models apart; fine-tuning them to tackle specific tasks and domains has become a standard practice, unlocking their true potential and elevating their performance to new heights. In this comprehensive guide, we’ll delve into the world of fine-tuning large language models, covering everything from the basics to advanced. This article was published as a part of the\xa0Data Science Blogathon. Pre-trained language models are large neural networks trained on vast corpora of text data, usually sourced from the internet. The training process involves

### Chunk text

In [32]:
max_chunk =500

In [33]:
Article = Article.replace('.','.<eos>')
Article = Article.replace('?','?<eos>')
Article = Article.replace('!','!<eos>')

In [34]:
sentences = Article.split('<eos>')
current_chunk = 0
chunk =[]

for sentence in sentences:
    if len(chunk) ==current_chunk +1:
        if len(chunk[current_chunk])+len(sentence.split(' '))<=max_chunk:
            chunk[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk +=1
            chunk.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunk.append(sentence.split(' '))

for chunk_id in range(len(chunk)):
    chunk[chunk_id] = ' '.join(chunk[chunk_id])

0


In [35]:
len(chunk)

6

### Summarize text

In [36]:
res = summarizer(chunk , max_length =120 ,min_length =30,do_sample =False)

In [37]:
res[0]

{'summary_text': ' Pre-trained language models are large neural networks trained on vast corpora of text data . These models are known for their ability to perform tasks such as text generation, sentiment classification, and language understanding at an impressive level of proficiency . Fine-tuning is like providing a finishing touch to these versatile models .'}

In [38]:
text = ' '.join([summary['summary_text'] for summary in res])

In [39]:
text

' Pre-trained language models are large neural networks trained on vast corpora of text data . These models are known for their ability to perform tasks such as text generation, sentiment classification, and language understanding at an impressive level of proficiency . Fine-tuning is like providing a finishing touch to these versatile models .  Fine-tuning involves training the pre-trained model on a smaller, task-specific dataset . This new dataset is labeled with examples relevant to the target task . By exposing the model to these labeled examples, it can adjust its parameters and internal representations to become well-suited for a specific task .  In this comprehensive guide, we will explore the concept of instruction fine-tuning and its implementation step-by-step . We will use GPT-3, a state-of-the-art language model, for this example . The instructions will guide the model’s sentiment analysis behavior .  Multitask instruction fine-tuning embraces a new paradigm by training la

### Output to text file

In [40]:
with open('blogsummary.txt','w') as f:
    f.write(text)